# Support Vector Machines (SVM)

In [1]:
# Imports
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load dataset
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [2]:
# Train SVM with different kernels
models = {
    'linear': SVC(kernel='linear', C=1.0, random_state=42),
    'rbf': SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42),
    'poly-3': SVC(kernel='poly', degree=3, C=1.0, gamma='scale', random_state=42)
}

results = {}
for name, clf in models.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = {
        'accuracy': acc,
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'report': classification_report(y_test, y_pred, target_names=iris.target_names)
    }

# Show accuracies
for name, res in results.items():
    print(f"\n=== Kernel: {name} ===")
    print("Accuracy:", res['accuracy'])
    print("Confusion Matrix:\n", res['confusion_matrix'])
    print("Classification Report:\n", res['report'])



=== Kernel: linear ===
Accuracy: 1.0
Confusion Matrix:
 [[10  0  0]
 [ 0 10  0]
 [ 0  0 10]]
Classification Report:
               precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00        10
   virginica       1.00      1.00      1.00        10

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30


=== Kernel: rbf ===
Accuracy: 0.9666666666666667
Confusion Matrix:
 [[10  0  0]
 [ 0  9  1]
 [ 0  0 10]]
Classification Report:
               precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      0.90      0.95        10
   virginica       0.91      1.00      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97  

### K-Nearest Neighbors (KNN)

In [3]:
from sklearn.neighbors import KNeighborsClassifier

# Try KNN with k=5
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn, target_names=iris.target_names))


Accuracy: 0.9333333333333333
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       0.83      1.00      0.91        10
   virginica       1.00      0.80      0.89        10

    accuracy                           0.93        30
   macro avg       0.94      0.93      0.93        30
weighted avg       0.94      0.93      0.93        30



In [5]:
from sklearn.pipeline import make_pipeline

def knn_acc(k=5, weights='uniform'):
    pipe = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=k, weights=weights))
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    return accuracy_score(y_test, y_pred)

Ks = [1, 3, 5, 7, 11, 15]
acc_uniform = {k: knn_acc(k, 'uniform') for k in Ks}
acc_distance = {k: knn_acc(k, 'distance') for k in Ks}

print("Uniform weights:", acc_uniform)
print("Distance weights:", acc_distance)


Uniform weights: {1: 0.9666666666666667, 3: 0.9333333333333333, 5: 0.9333333333333333, 7: 0.9666666666666667, 11: 0.9666666666666667, 15: 0.9666666666666667}
Distance weights: {1: 0.9666666666666667, 3: 0.9666666666666667, 5: 0.9666666666666667, 7: 1.0, 11: 1.0, 15: 1.0}


In [6]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Synthetic regression dataset
rng = np.random.RandomState(42)
X_reg = np.linspace(0, 10, 200).reshape(-1, 1)
y_reg = np.sin(X_reg).ravel() + rng.normal(scale=0.2, size=200)

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

def knn_reg_mse(k=5, weights='uniform'):
    pipe = make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=k, weights=weights))
    pipe.fit(X_train_r, y_train_r)
    y_pred_r = pipe.predict(X_test_r)
    return mean_squared_error(y_test_r, y_pred_r)

Ks_reg = [1, 3, 5, 7, 11, 15]
mse_uniform = {k: knn_reg_mse(k, 'uniform') for k in Ks_reg}
mse_distance = {k: knn_reg_mse(k, 'distance') for k in Ks_reg}

print("Uniform MSE:", mse_uniform)
print("Distance MSE:", mse_distance)


Uniform MSE: {1: 0.06313331077295618, 3: 0.04844225621315464, 5: 0.038633805316679, 7: 0.035922549335748996, 11: 0.035521106325741035, 15: 0.03452551667268621}
Distance MSE: {1: 0.06313331077295618, 3: 0.047083663937253196, 5: 0.039386279139711544, 7: 0.037833976270262325, 11: 0.03639689779904827, 15: 0.035861438825264116}
